# GPT-3.5-Turbo on GSM8K

In [1]:
!pip install openai tqdm datasets llmlingua tenacity

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 10.3 MB/s eta 0:00:00
  Using cached nvidia

In [2]:
from openai import OpenAI
import re
import time

import numpy as np

from tqdm import tqdm
from datasets import load_dataset
from llmlingua import PromptCompressor


In [1]:
client = OpenAI(
    # This is the default and can be omitted
    api_key="sk-",
)
llm_lingua = PromptCompressor()

NameError: name 'OpenAI' is not defined

In [5]:
gsm8k = load_dataset('gsm8k', 'main')
validation_index = np.load('validation_index.npy')
validation_data = gsm8k['train'].select(validation_index)
gsm8k_test = gsm8k['test']

In [6]:
gsm8k['train'][0]['question']

'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?'

In [7]:
gsm8k_test = gsm8k['test']

In [8]:
prompt_complex = open('prompt_hardest.txt').read()

In [26]:
print(prompt_complex[:1000])

Question: Angelo and Melanie want to plan how many hours over the next week they should study together for their test next week. They have 2 chapters of their textbook to study and 4 worksheets to memorize. They figure out that they should dedicate 3 hours to each chapter of their textbook and 1.5 hours for each worksheet. If they plan to study no more than 4 hours each day, how many days should they plan to study total over the next week if they take a 10-minute break every hour, include 3 10-minute snack breaks each day, and 30 minutes for lunch each day?
Let's think step by step
Angelo and Melanie think they should dedicate 3 hours to each of the 2 chapters, 3 hours x 2 chapters = 6 hours total.
For the worksheets they plan to dedicate 1.5 hours for each worksheet, 1.5 hours x 4 worksheets = 6 hours total.
Angelo and Melanie need to start with planning 12 hours to study, at 4 hours a day, 12 / 4 = 3 days.
However, they need to include time for breaks and lunch. Every hour they want 

In [27]:
from tenacity import (
    retry,
    stop_after_attempt,
    wait_chain,
    wait_fixed
)

@retry(wait=wait_chain(*[wait_fixed(3) for i in range(3)] +
                       [wait_fixed(5) for i in range(2)] +
                       [wait_fixed(10)]))
def completion_with_backoff(**kwargs):
    return client.chat.completions.create(**kwargs)

In [28]:
def compress_txt(q, max_token=300):
    return llm_lingua.compress_prompt(q, instruction="", question="", target_token=max_token)['compressed_prompt']

In [34]:
prompt_q = compress_txt(prompt_complex, 500) + '\nQuestion: ' + gsm8k_test[1]['question'] + '\n'

OutOfMemoryError: CUDA out of memory. Tried to allocate 24.00 MiB. GPU 

In [35]:
print(prompt_q[-500:])

ts.
So, there are 3 x 200 = 600 males.
And there are 2 x 200 = 400 females.
Hence, 600 x 2/3 = 400 males play basketball.
And 400 x 1/5 = 80 females play basketball.
A total of 400 + 80 = 480 students play basketball.
Therefore, 1000 - 480 = 520 do not like to play basketball.
The percentage of the school that do not like to play basketball is 520/1000 * 100 = 52
The answer is 52

Question: A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?



In [14]:
response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "Follow the given examples and answer the question."
        },
        {
          "role": "user", "content": prompt_q
        },
    ],
    model="gpt-3.5-turbo",
)

In [15]:
print(response.choices[0].message.content)

Since the robe takes 2 bolts of blue fiber and half that amount of white fiber, it takes 2 + 2/2 = 2 + 1 = 3 bolts in total.  
The answer is 3.
